In [70]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

#username = "userAdmin"
username = "aacuser"
password = "aac"
shelter = AnimalShelter(username, password)


# must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'flex'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H3(html.Center("Hitchhiker's Guide to the Galaxy: Towel Search and Rescue Animals")),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        
        # upload data
        data=df.to_dict('records'),        
        
        # cells
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '100px',
            'maxWidth': '200px',
            'padding': '5px 10px',
            'textAlign': 'left'
        },
        style_header={
            'textAlign': 'center',
            'backgroundColor':'beige',
            'fontWeight': 'bold'          
        },
        
        # rows and columns
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #column_selectable=False,
        #row_selectable=True,
        #row_deletable=True,
        
        # editting
        editable=False,
                
        # filter & sort
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
      
        # page setup
        page_action="native",
        page_current= 0,
        page_size= 25
        
    ),
    html.Br(),
    html.Hr(),
    
    html.Div(id='map-div', style={'display':'flex'}),
    html.Center(html.B(html.H2('SNHU CS-340 Map'))),
    html.Hr(),
    html.Br(), 
    
    html.Div(
        id='map-id',
        className='col s12 m6',
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# highlights cell on data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# map to display location of pets
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ] 
    
    
#!!do not remove!!
app

Connecting to the MongoDB
